In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mnist = pd.read_csv("./Data/train.csv")
mnist.head()

In [ ]:
x_data = mnist.drop("label", axis=1)
x_data.head()

In [ ]:
# normalize pixel values
x_data = x_data / 255
x_data.iloc[:1,:].values.max()

In [ ]:
y_labels = mnist["label"]
y_labels.head()

In [ ]:
y_labels = pd.get_dummies(y_labels)

In [ ]:
y_labels.head()

In [ ]:
x_data = np.array(x_data.values)
x_data.shape

In [ ]:
x_train = x_data[:37000]
x_train.shape

In [ ]:
x_validation = x_data[37000:]
x_validation.shape

In [ ]:
y_train = y_labels[:37000].values
y_train.shape

In [ ]:
y_validation = y_labels[37000:].values
y_validation.shape

In [ ]:
# Weight initialization function
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return (tf.Variable(init_random_dist))

In [ ]:
# Bias initialization function
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [ ]:
# 2d convolution function which is already performed by tf
def conv2d(x, W):
    # x => input tensor ==> [batch, H, W, Channels]
    # W => kernel => [filter height, filter width, # of channels in, # channels out]
    return tf.nn.conv2d(x, W, strides=[1,1,1,1],padding="SAME")
# padding with SAME adds zeros to the end
# Strides is how you want to move in the whole thing [batch, height, width, channel]

In [ ]:
# Pooling function in this case max pooling which gets the max value and is a 2x2 kernel
def max_pool_2by2(x):
    # x=> input => [batch, h, w, c]
    return (tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME"))
# Ksize = size of the window which makces the pooling [batch, h, w, c] => here 1 is like pasing all 2 is reducing
    # We only want to reduce the height and width of the image, that is why we use 
    # [1 in batch, 2 in height, 2 in width, 1 in channel]
#  Stride here is [1, 2, 2, 1] because we want to shorten the image so we jump 2 by 2 pixels in the image

In [ ]:
# convolutional layer function
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]]) 
    # we take the third one because that is the number of channels we are using
    # is not really intuitive but the channels is like the number of features
    # first is going to be 1 then more
    return (tf.nn.relu(conv2d(input_x, W)+b))

In [ ]:
# normal layaer (fully connected layer)
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1]) 
    # we use the index one beacause 0 is the batch size and 1 is the number of nodes
    W = init_weights([input_size, size])
    b = init_bias([size])
    return (tf.matmul(input_layer, W)+b)
# simple weighted sum

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [ ]:
# image layer => change the image to a 28 by 28 grid again
x_image = tf.reshape(x, [-1, 28, 28, 1])
# the -1 is a special value in the reshape function which means that the value is going to be computed so that 
# everythinh else in the array remains constant
#If one component of shape is the special value -1, the size of that dimension is computed so that 
# the total size remains constant

In [ ]:
# First convolutional layer
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32]) 
#we are using a 5x5 filter with 1 channel in and 32 features out
# 32 feaures out means that we are going to apply the filter to the whole image 32 times
# First pooling layer
convo_1_pooling = max_pool_2by2(convo_1)

In [ ]:
# Convolutional layer 2
convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])
# 32 because we are getting that many features from the past convo and 64 is the number we want to get out of it
# remember we get those features by the matrix multiplication
convo_2_pooling = max_pool_2by2(convo_2)

In [ ]:
# now we flatten the result
# it is important to notice that we have a 7by7 image right now because the convolution has padding,
# which makes the image to just go throuhg the layer withou loosing its size
# So the only thing changing the size are the pooling layer which diivide the width and height by 2 and then again by 2
convo_2_flat = tf.reshape(convo_2_pooling,[-1, 7*7*64])
# 7*7 because of the width and height and 64 because of the channels

In [ ]:
# now we put the normal full layer
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))
# 1024 is just the number of nodes we want on the next layer, which is just based in the model we are creating

In [ ]:
# Regularization function which is dropout
hold_prob = tf.placeholder(tf.float32) # just keeps our probability of a node staying
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)
# tf function only need the probability and the layer you want to make dropout to

In [ ]:
# final layer to reduce to number of results
y_pred = normal_full_layer(full_one_dropout, 10)
# we only want ten results one for each digit

In [ ]:
# error funcitons
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred))

In [ ]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar("accuracy", acc)

In [ ]:
#variable initializer
init = tf.global_variables_initializer()

In [ ]:
#savers
saver = tf.train.Saver()

In [ ]:
#summaries for tensorboard
merged_summary_op = tf.summary.merge_all() # this will get all the summaries from the graph and join them

In [ ]:
steps = 1000
batch_size = 128

with tf.Session() as sess:
    sess.run(init)
    
    # to use tensorboard you need to have a file writer to keep the logs
    train_writer = tf.summary.FileWriter("./tensorboard/train", graph=tf.get_default_graph())
    validation_writer = tf.summary.FileWriter("./tensorboard/validation", graph=tf.get_default_graph())
    
    for i in range(steps + 1):
        
        rand_ind = np.random.randint(len(x_train), size=batch_size)
        # Here we add hold_prob to our feed dictionary because it is a placeholder
        feed = {x:x_train[rand_ind], y_true:y_train[rand_ind], hold_prob:0.5}
        sess.run(train, feed_dict=feed)
        
        
        # When runnining you need to append logs for tensorboard
        if i%50 == 0:
            summary = sess.run(merged_summary_op,
                                      {x:x_train[rand_ind], y_true:y_train[rand_ind], hold_prob:1.0})
            train_writer.add_summary(summary, i)
            train_writer.flush()
            summary = sess.run(merged_summary_op,
                                      {x:x_validation, y_true:y_validation, hold_prob:1.0})
            validation_writer.add_summary(summary, i)
            validation_writer.flush()
            print("checkpoints #{}".format(i))
    
    saver.save(sess, "cnn_model/cnn.ckpy")

### Predictions


In [ ]:
test_data = pd.read_csv("./Data/test.csv")
test_data.head()

In [ ]:
test_data.iloc[0:1,:].values.max()

In [ ]:
test_data = test_data / 255

In [ ]:
test_data.iloc[0:1,:].values.max()

In [ ]:
test_data = test_data.values
test_data

In [ ]:
test_data.shape

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "cnn_model/cnn.ckpy")
    
    prediction = y_pred
    predictions = sess.run(prediction, feed_dict={x:test_data, hold_prob:1.0})
    result = []
    decode = tf.argmax(predictions, axis=1)
    result = sess.run(decode)
    print (result)

In [ ]:
result_df = pd.DataFrame(result, columns=["Label"], dtype=int)
result_df.index +=1

In [ ]:
result_df.index.name = "ImageId"

In [ ]:
result_df.head()

In [ ]:
result_df.to_csv("./result_for_cnn.csv")

In [ ]:
#for tensorboard on terminal home directory
# tensorboard --logdir=./tensorboard 